In [2]:
!pip install spacy

  Using cached spacy_legacy-3.0.12-py2.py3-none-any.whl.metadata (2.8 kB)
  Using cached spacy_loggers-1.0.5-py3-none-any.whl.metadata (23 kB)
  Using cached preshed-3.0.9-cp312-cp312-win_amd64.whl.metadata (2.2 kB)
  Using cached wasabi-1.1.3-py3-none-any.whl.metadata (28 kB)
  Using cached catalogue-2.0.10-py3-none-any.whl.metadata (14 kB)
  Using cached weasel-0.4.1-py3-none-any.whl.metadata (4.6 kB)
  Using cached annotated_types-0.7.0-py3-none-any.whl.metadata (15 kB)
  Using cached typing_extensions-4.12.2-py3-none-any.whl.metadata (3.0 kB)
  Using cached confection-0.1.5-py3-none-any.whl.metadata (19 kB)
  Using cached shellingham-1.5.4-py2.py3-none-any.whl.metadata (3.5 kB)
  Using cached markdown_it_py-3.0.0-py3-none-any.whl.metadata (6.9 kB)
  Using cached mdurl-0.1.2-py3-none-any.whl.metadata (1.6 kB)
   ---------------------------------------- 0.0/11.8 MB ? eta -:--:--
   ---------------------------------------  11.5/11.8 MB 71.8 MB/s eta 0:00:01
   ------------------------

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.5.1 requires sympy==1.13.1; python_version >= "3.9", but you have sympy 1.13.3 which is incompatible.


In [3]:
import torch
import spacy
from transformer import Transformer 
import torch.optim as optim
import os
import json

In [6]:
import spacy

nlp_en = spacy.load('en_core_web_sm')
nlp_fr = spacy.load('fr_core_news_sm')


OSError: [E050] Can't find model 'en_core_web_sm'. It doesn't seem to be a Python package or a valid path to a data directory.

In [5]:
nlp_en = spacy.load('en_core_web_sm')
nlp_fr = spacy.load('fr_core_news_sm')

OSError: [E050] Can't find model 'en_core_web_sm'. It doesn't seem to be a Python package or a valid path to a data directory.

In [16]:
# Pre Process with punctuatuion and spaces
def preprocess_text_spacy(text, nlp_model):

    doc = nlp_model(text)
    tokens = []
    for token in doc:
        if not token.is_punct and not token.is_space:
            tokens.append(token.text.lower())
    return tokens

# Tokens to IDs
def tokens_to_ids(tokens, vocab):
    return [vocab.get(token, vocab['<UNK>']) for token in tokens]

In [7]:
with open('src_vocab.json', 'r') as src_file:
    src_vocab = json.load(src_file)

with open('tgt_vocab.json', 'r') as tgt_file:
    tgt_vocab = json.load(tgt_file)

In [8]:
src_vocab_size = len(src_vocab)
tgt_vocab_size = len(tgt_vocab)
d_model = 512
heads = 8
num_layers = 6
d_ff = 2048
max_seq_length = 100 
dropout = 0.1
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model = Transformer(src_vocab_size, tgt_vocab_size, d_model, heads, num_layers, d_ff, max_seq_length, dropout).to(device)

In [9]:
model

Transformer(
  (encoder_embedding): Embedding(16317, 512)
  (decoder_embedding): Embedding(29592, 512)
  (positional_encoding): PositionalEncoding()
  (encoder_layers): ModuleList(
    (0-5): 6 x EncoderLayer(
      (self_attn): MultiHeadAttention(
        (W_v): Linear(in_features=512, out_features=512, bias=True)
        (W_k): Linear(in_features=512, out_features=512, bias=True)
        (W_q): Linear(in_features=512, out_features=512, bias=True)
        (W_o): Linear(in_features=512, out_features=512, bias=True)
      )
      (feed_forward): PositionWiseFeedForward(
        (fcnn1): Linear(in_features=512, out_features=2048, bias=True)
        (fcnn2): Linear(in_features=2048, out_features=512, bias=True)
        (relu): ReLU()
      )
      (norm1): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
      (norm2): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
  )
  (decoder_layers): ModuleList(
    (0-5): 6 x DecoderLa

In [10]:
#             ignore index = 0-> Handing Padding Tokens
criterion = torch.nn.CrossEntropyLoss(ignore_index=0)

optimizer = optim.Adam(model.parameters(), lr=0.0001)

In [11]:
model = torch.load('Model.pth', map_location=torch.device('cpu'))

C:\Users\CT_USER\AppData\Local\Temp\ipykernel_20396\3011260972.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model = torch.load('Model.pth', map_location=torch.device(

In [89]:
def predict(model, src_tensor, max_len, device, id_to_word):
    model.eval()
    tgt_tokens = [src_vocab['<SOS>']]  
    tgt_tensor = torch.tensor([tgt_tokens]).to(device)
    
    predictions = []
 
    with torch.no_grad():
        for _ in range(max_len):
            output = model(src_tensor, tgt_tensor)
            next_token = output.argmax(dim=-1)[:, -1].item()
            predictions.append(next_token)
           
            tgt_tokens.append(next_token)
            tgt_tensor = torch.tensor([tgt_tokens]).to(device)  
 
    predicted_words = [id_to_word.get(token_id, '<UNK>') for token_id in predictions]
    predicted_sentence = ' '.join(predicted_words)
    return predicted_sentence

In [99]:
def generate_predicted_sentence(model, src_sentence, src_vocab, tgt_vocab, preprocess, nlp, predict, device, max_len=None):
    num = src_sentence.count(' ')+2
    id_to_word = {id_: word for word, id_ in tgt_vocab.items()}
    src_tensor = torch.tensor(tokens_to_ids(preprocess(src_sentence, nlp), src_vocab)).unsqueeze(0).to(device)
    predicted_sentence = predict(model, src_tensor, max_len=num, device=device, id_to_word=id_to_word)
    a=0
    if (predicted_sentence.__contains__('<UNK>')):
        a=a+1
    predicted_sentence=predicted_sentence.replace('<UNK>','')
    
    def get_prediction(predicted_sentence, num, a):
        words = predicted_sentence.split()  
        return ' '.join(words[:num + a]) 
    ans=get_prediction(predicted_sentence, num, a)
    return ans

In [100]:
src_sentence = "I am going there"
predicted_sentence = generate_predicted_sentence(model=model,src_sentence=src_sentence,src_vocab=src_vocab,tgt_vocab=tgt_vocab,preprocess=preprocess_text_spacy,
                                                    nlp=nlp_en,predict=predict,device=device)
print("Input Sentence:" ,src_sentence)
print(f'Predicted translation: {predicted_sentence}')

Input Sentence: I am going there
Predicted translation: je vais y aller


In [101]:
src_sentence = "I am having fun on Friday"
predicted_sentence = generate_predicted_sentence(model=model,src_sentence=src_sentence,src_vocab=src_vocab,tgt_vocab=tgt_vocab,preprocess=preprocess_text_spacy,
                                                    nlp=nlp_en,predict=predict,device=device)
print("Input Sentence:" ,src_sentence)

print(f'Predicted translation: {predicted_sentence}')

Input Sentence: I am having fun on Friday
Predicted translation: bien vendredi je amuse vendredi vendredi


In [102]:
src_sentence = "I am here"
predicted_sentence = generate_predicted_sentence(model=model,src_sentence=src_sentence,src_vocab=src_vocab,tgt_vocab=tgt_vocab,preprocess=preprocess_text_spacy,
                                                    nlp=nlp_en,predict=predict,device=device)
print("Input Sentence:" ,src_sentence)
print(f'Predicted translation: {predicted_sentence}')

Input Sentence: I am here
Predicted translation: je suis ici


In [103]:
src_sentence = "I came from college"
predicted_sentence = generate_predicted_sentence(model=model,src_sentence=src_sentence,src_vocab=src_vocab,tgt_vocab=tgt_vocab,preprocess=preprocess_text_spacy,
                                                    nlp=nlp_en,predict=predict,device=device)
print("Input Sentence:" ,src_sentence)
print(f'Predicted translation: {predicted_sentence}')

Input Sentence: I came from college
Predicted translation: un collège de


In [104]:
src_sentence = "My name is Rohit"
predicted_sentence = generate_predicted_sentence(model=model,src_sentence=src_sentence,src_vocab=src_vocab,tgt_vocab=tgt_vocab,preprocess=preprocess_text_spacy,
                                                    nlp=nlp_en,predict=predict,device=device)
print("Input Sentence:" ,src_sentence)
print(f'Predicted translation: {predicted_sentence}')

Input Sentence: My name is Rohit
Predicted translation: mon nom est bleu


In [105]:
src_sentence = "I lives in India"
predicted_sentence = generate_predicted_sentence(model=model,src_sentence=src_sentence,src_vocab=src_vocab,tgt_vocab=tgt_vocab,preprocess=preprocess_text_spacy,
                                                    nlp=nlp_en,predict=predict,device=device)
print("Input Sentence:" ,src_sentence)
print(f'Predicted translation: {predicted_sentence}')

Input Sentence: I lives in India
Predicted translation: inde je suis inde


In [109]:
src_sentence = "I coding right now"
predicted_sentence = generate_predicted_sentence(model=model,src_sentence=src_sentence,src_vocab=src_vocab,tgt_vocab=tgt_vocab,preprocess=preprocess_text_spacy,
                                                    nlp=nlp_en,predict=predict,device=device)
print("Input Sentence:" ,src_sentence)
print(f'Predicted translation: {predicted_sentence}')

Input Sentence: I coding right now
Predicted translation: tout de suite je suis


In [110]:
src_sentence = "I am playing cricket on field"
predicted_sentence = generate_predicted_sentence(model=model,src_sentence=src_sentence,src_vocab=src_vocab,tgt_vocab=tgt_vocab,preprocess=preprocess_text_spacy,
                                                    nlp=nlp_en,predict=predict,device=device)
print("Input Sentence:" ,src_sentence)
print(f'Predicted translation: {predicted_sentence}')

Input Sentence: I am playing cricket on field
Predicted translation: ai du cricket sur le champ


In [111]:
src_sentence = "I am playing cricket"
predicted_sentence = generate_predicted_sentence(model=model,src_sentence=src_sentence,src_vocab=src_vocab,tgt_vocab=tgt_vocab,preprocess=preprocess_text_spacy,
                                                    nlp=nlp_en,predict=predict,device=device)
print("Input Sentence:" ,src_sentence)
print(f'Predicted translation: {predicted_sentence}')

Input Sentence: I am playing cricket
Predicted translation: ai du cricket je
